Following HuggingFace SpeechT5 tutorial to combine ASR and TTS (https://huggingface.co/blog/speecht5)


In [38]:
!pip install -q sentencepiece
!pip install -q torchaudio 
!pip install -q transformers
!pip install -q soundfile

# there's probably a better library for playing audio 
!pip install -q simpleaudio 

In [4]:
!ls ../data

2086-149220-0033.wav


In [8]:
import soundfile as sf

input_sound = sf.read("../data/2086-149220-0033.wav")

In [9]:
input_sound 

(array([0.00000000e+00, 9.15527344e-05, 9.15527344e-05, ...,
        1.22070312e-04, 1.22070312e-04, 1.22070312e-04]),
 16000)

In [11]:
from transformers import pipeline
asr = pipeline(task="automatic-speech-recognition", model="microsoft/speecht5_asr")

In [13]:
text = asr(input_sound[0])

/Users/iskander/miniconda3/lib/python3.10/site-packages/transformers/generation/utils.py:1349: UserWarning: Using `max_length`'s default (450) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [16]:
text

{'text': "well i don't wish to see it any more observed febric turning away her eyes it is certainly very like the old portrait"}

In [18]:
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech

tts_processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
tts_model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")

tts_inputs = tts_processor(text=text['text'], return_tensors="pt");
tts_inputs

{'input_ids': tensor([[ 4, 20,  5, 15, 15,  4, 10,  4, 14,  8,  9, 31,  6,  4, 20, 10, 12, 11,
          4,  6,  8,  4, 12,  5,  5,  4, 10,  6,  4,  7,  9, 22,  4, 18,  8, 13,
          5,  4,  8, 25, 12,  5, 13, 27,  5, 14,  4, 19,  5, 25, 13, 10, 17,  4,
          6, 16, 13,  9, 10,  9, 21,  4,  7, 20,  7, 22,  4, 11,  5, 13,  4,  5,
         22,  5, 12,  4, 10,  6,  4, 10, 12,  4, 17,  5, 13,  6,  7, 10,  9, 15,
         22,  4, 27,  5, 13, 22,  4, 15, 10, 28,  5,  4,  6, 11,  5,  4,  8, 15,
         14,  4, 24,  8, 13,  6, 13,  7, 10,  6,  2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [20]:
import torch
from transformers import SpeechT5HifiGan
from datasets import load_dataset

# load vector describing speaker voice
speaker_embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
speaker_embeddings = torch.tensor(speaker_embeddings_dataset[7306]["xvector"]).unsqueeze(0)

# get a vocoder model to generate final sound
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")
output_speech = tts_model.generate_speech(tts_inputs["input_ids"], speaker_embeddings, vocoder=vocoder)

Found cached dataset cmu-arctic-xvectors (/Users/iskander/.cache/huggingface/datasets/Matthijs___cmu-arctic-xvectors/default/0.0.1/a62fea1f9415e240301ea0042ffad2a3aadf4d1caa7f9a8d9512d631723e781f)


In [21]:
import soundfile as sf
sf.write("round-trip-output.wav", output_speech.numpy(), samplerate=16000)

In [37]:
!pip install simpleaudio 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 12.0 MB/s eta 0:00:00m eta 0:00:010:01
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [85 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build/lib.macosx-11.1-arm64-cpython-310
      creating build/lib.macosx-11.1-arm64-cpython-310/simpleaudio
      copying simpleaudio/__init__.py -> build/lib.macosx-11.1-arm64-cpython-310/simpleaudio
      copying simpleaudio/shiny.py -> build/lib.macosx-11.1-arm64-cpython-310/simpleaudio
      copying simpleaudio/functionchecks.py -> build/lib.macosx-11.1-arm64-cpython-310/simpleaudio
      creating build/lib.macosx-11.1-arm64-cpython-310/simpleaudio/test_audio
      copying simpleaudio/test_audio/c.wav -> build/lib.macosx-11.1-arm64-cpython-310/simpleaudio/test_audio
      copying simpleaudio/test_audio/e.

Failed to build simpleaudio
  Running setup.py install for simpleaudio ... done
  DEPRECATION: simpleaudio was installed using the legacy 'setup.py install' method, because a wheel could not be built for it. pip 23.1 will enforce this behaviour change. A possible replacement is to fix the wheel build issue reported above. Discussion can be found at https://github.com/pypa/pip/issues/8368


In [52]:
import simpleaudio 
import numpy as np
# simpleaudio expects 16-bit integer values for wave height
def play(single_channel_float_waveform, sampling_rate=16000):
    # normalize to 16-bit range
    min_int = -32768
    max_int =  32767
    int_range = max_int - min_int
    normalized_waveform = single_channel_float_waveform - single_channel_float_waveform.min()
    normalized_waveform /= normalized_waveform.max()
    int32_waveform_from_0 = (normalized_waveform * int_range).astype(np.int32)
    int32_waveform_from_min = int32_waveform_from_0 + min_int
    int16_waveform = int32_waveform_from_min.astype(np.int16)
    play_obj = simpleaudio.play_buffer(int16_waveform, 1, 2, 16000)
    play_obj.wait_done()

In [55]:
play(input_sound[0])

In [56]:
play(output_speech.numpy())